## Build Simple LLM Application with LCEL

In this quickstart we'll show you how to build a simple LLm application with Langchain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain- a lot of features can be built with just some prompting and an LLM call!

High level overview of:

Using language models.
using PromptTemplates and OutputParsers.
Using LangChain Expression Language (LCEL) to chain components together.
Debugging and tracing your application using LangSmith.




In [4]:
### OPen AI API key and OPen source models--Llama3, Gemma2, Mistral--Groq

import os
from dotenv import load_dotenv

load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")


groq_api_key=os.getenv("GROQ_API_KEY")


In [7]:
from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023DA1189B40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023DA3BA16C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage
messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello, how are you?")
]

result= model.invoke(messages)
result.content

'Bonjour, comment allez-vous ? \n\n\n(pronounced: bohn-zhoor, koh-mahn a-lay voo)\n'

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser= StrOutputParser()
parser.invoke(result)


'Bonjour, comment allez-vous ? \n\n\n(pronounced: bohn-zhoor, koh-mahn a-lay voo)\n'

In [16]:
### Prompt Template
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the follwing into {language}:"
prompt= ChatPromptTemplate.from_messages([("system", generic_template), ("user", "{text}")]
                                         )


ChatPromptValue(messages=[SystemMessage(content='Translate the follwing into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [20]:
result= prompt.invoke({"language": "French", "text": "Hello, how are you?"})

In [21]:
result.to_messages()

[SystemMessage(content='Translate the follwing into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [22]:
chain=prompt |model|parser
chain.invoke({"language": "French", "text": "Hello, how are you?"})

'The most common way to say "Hello, how are you?" in French is:\n\n**Bonjour, comment allez-vous ?** \n\nHere\'s a breakdown:\n\n* **Bonjour** means "Hello"\n* **Comment** means "How"\n* **allez-vous**  is the formal way to say "are you?" \n\nFor a more informal greeting, you could say:\n\n**Salut, ça va ?** \n\n* **Salut** means "Hi" or "Hello"\n* **ça va** is a casual way to say "how are you?"\n\n\nLet me know if you\'d like to learn any other French phrases!\n'